<h1>Get the files</h1>

In [1]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
from collections import Counter
import glob, os, sys, re
import numpy as np
import pandas as pd
from pyspark.sql import SQLContext, Row
from pyspark.sql.functions import *
base_dir = "/Users/jgartner/Desktop/gt/dumps/top_txt/"
txt_files = glob.glob(base_dir+"*.*")

In [2]:
print(len(txt_files))

txt_files[:10]

6364


['/Users/jgartner/Desktop/gt/dumps/top_txt/0000_convicted_pedophile.txt',
 '/Users/jgartner/Desktop/gt/dumps/top_txt/000webhost.com.txt',
 '/Users/jgartner/Desktop/gt/dumps/top_txt/000webhost.txt',
 '/Users/jgartner/Desktop/gt/dumps/top_txt/01ANdR3wz.txt',
 '/Users/jgartner/Desktop/gt/dumps/top_txt/03Ah4w7B1tch3z.txt',
 '/Users/jgartner/Desktop/gt/dumps/top_txt/0411e32a7b59947fa789f7da71a81a34539e0f1fbcf1ca00c620c0bac93a3b80.txt',
 '/Users/jgartner/Desktop/gt/dumps/top_txt/0b25da9661a5b00d67c09a3a6cf2cc256da3ccebaeaa43474ec2bbca1f4ffffb.txt',
 '/Users/jgartner/Desktop/gt/dumps/top_txt/0be143fe99756aa0796412f9d1021785cdaf26ab49ef6bd0648dfd4bf9dc1dcd.txt',
 '/Users/jgartner/Desktop/gt/dumps/top_txt/0c65ace3636ec38400ab28cf4076440bb61498a5a409234117ff859062ea8323.txt',
 '/Users/jgartner/Desktop/gt/dumps/top_txt/0day_aka_0seconds.txt']

<h1>File Sizes</h1>

In [3]:
file_sizes =[np.log10(x) for x in filter(lambda x: x != 0, [os.stat(x).st_size for x in  txt_files])]

In [4]:
data = [
    go.Histogram(
        x=file_sizes,
        marker=dict(
            color='green',
            line=dict(
                color='black',
                width=0
            )
        ),
    )
]



layout = go.Layout(
    title='Log10 File Sizes',
    xaxis=dict(
        title='Value'
    ),
    yaxis=dict(
        title='Count'
    ),
    barmode='overlay',
    bargap=0.25,
    bargroupgap=0.3
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

<h1>Hack Attribution</h1>
<p>We will parse the begining lines of the Dox file, looking for attribution information.  If any is found, we store by hacker.</p>

In [5]:
def attribute_hack(file):
    f0 = open(file, "r")
    n_lines = 0
    b_next=False
    try:
        for line in f0:
            #line = re.sub('[\s]', ' ', line, flages=re.UNICODE)
            line = re.sub('[^\w]',' ',line.lower(),flags=re.UNICODE) 
            words = line.split()
            if b_next:
                return (line, file)
            if 'by' in words:
                ind = words.index('by')
                if ind+1 == len(words):
                    b_next=True
                else:
                    if len(words[ind+1:]) < 4:
                        return (' '.join(words[ind+1:]), file)
            n_lines+=1
            if n_lines==30:
                return None
    except:
        print("File:", file)
        print("Unexpected error:", sys.exc_info()[0])
        print("Line:", line)
        return None
    return None

In [6]:
hackers = list(filter(lambda x: x!=None, [attribute_hack(x) for x in txt_files]))
#len(list(filter(lambda x: x==-1, [attribute_hack(x) for x in txt_files])))

In [7]:
len(hackers)

537

In [8]:
rdd0 = sc.parallelize(hackers)

In [9]:
com = rdd0.map(lambda x: (x[0], 1)).reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[1]).filter(lambda x: x[0] != '  ').filter(lambda x: x[1]> 2).collect()

In [10]:
sorted(com, key=lambda x: x[1], reverse=True)

[('an0nblank', 15),
 ('obnoxious', 11),
 ('vypor', 7),
 ('cock security', 7),
 ('avvo', 6),
 ('cgod', 6),
 ('vindecky', 5),
 ('h0atzin', 4),
 ('master_cat', 4),
 ('napalmf re', 3),
 ('friend 1 463468', 3),
 ('viruz mafia', 3),
 ('quot bill quot', 3),
 ('bwa kank', 3),
 ('kank', 3),
 ('sonnyspooks', 3),
 ('welfare', 3)]

In [17]:
cs = list(filter(lambda x: x[0] == "cock security", hackers))

In [18]:
cs

[('cock security', '/Users/jgartner/Desktop/gt/dumps/top_txt/Ann_Coulter.txt'),
 ('cock security',
  '/Users/jgartner/Desktop/gt/dumps/top_txt/John_Donahoe_Ebay_CEO.txt'),
 ('cock security',
  '/Users/jgartner/Desktop/gt/dumps/top_txt/koch_brothers.txt'),
 ('cock security',
  '/Users/jgartner/Desktop/gt/dumps/top_txt/mayhem_miller.txt'),
 ('cock security', '/Users/jgartner/Desktop/gt/dumps/top_txt/oprah.txt'),
 ('cock security',
  '/Users/jgartner/Desktop/gt/dumps/top_txt/Raymond_W_Kelly_NYPD_Commissioner.txt'),
 ('cock security', '/Users/jgartner/Desktop/gt/dumps/top_txt/reallyrick.txt')]

<h2>Classify Txt documents based on simple rules</h2>
<p>Before auto-ingest and other tasks can be accomplished, try to determine file type.</p>

In [ ]:
def is_sql(line):
    key = " ".join(line.split()[:2])
    if key in ["INSERT INTO", "CREATE TABLE"]:
        return True
    return False

def csv_potential(line, char):
    return len(line.split(char))


def move_to_folder(file, debug=False):
    #return types: 
    #0 - sql
    #1 - csv/tsv
    #2 - hash
    #3 - space delimited/ambiguous
    #4 - unstructured/dox
    f0 = open(file, "r")
    n_lines = 0
    delim = [",", ";", "|",":", "\t", " "]
    d_csv = {}
    for d in delim:
        d_csv[d] = []
    try:
        for line in f0:
            if is_sql(line):
                return 0 #0 type = sql
            for d in delim:
                d_csv[d].append(csv_potential(line, d))
            n_lines += 1
            if n_lines == 150:
                break
        
        if n_lines < 5:
            return 4
        for k, v in d_csv.iteritems():
            if debug:
                print k, v
            s_vs = sorted(Counter(v).iteritems(), key=lambda x: x[1], reverse=True)
            m_v = s_vs[0][0]
            sm_v = s_vs[1][0]
            if m_v==1 or m_v==0:
                continue
            if float(len(filter(lambda x: x==m_v, v)))/len(v) > .85:
                if k in [",", "\t", "|", ";"]:
                    return 1
                elif k == ":" and m_v == 2:
                    return 2
                elif k == ":":
                    return 1
                elif k == " ":
                    return 3
            if float(len(filter(lambda x: x==m_v, v)))/len(v) > .4 and float(len(filter(lambda x: x==s_v, v)))/len(v) > .4:
                return 0
        
        return 4
    except:
        print("File:", file)
        print("Unexpected error:", sys.exc_info()[0])
        print("Line:", line)
        return -1
    

In [ ]:
sub_dirs = ['sql_files', 'csv_files', 'hash_files', 'ss_files', 'dox_files']

In [ ]:
def move_file(file):
    tt = file.split("/")
    new_dir = "/".join(tt[:-1]) + "/" + sub_dirs[move_to_folder(file)] + "/" + tt[-1]
    print new_dir
    os.rename(file, new_dir)

In [ ]:
for file in txt_files:
    move_file(file)

In [ ]:
counts = []
for d in sub_dirs:
    counts.append(len(os.listdir(base_dir+d)))

In [ ]:
counts

In [ ]:
file_sizes = map(lambda x: sum(os.path.getsize(base_dir + x + "/" + f) for f in os.listdir(base_dir+x)), sub_dirs)


fig = {
    'data': [
        {
            'labels': ['SQL Dumps', 'CSV Files', 'Hash Files', 'Space Separated Files', 'Dox Files'],
            'values': counts,
            'type': 'pie',
            'domain': {'x': [0, .48]},
            'hole': .4,
            'hoverinfo':'label+percent',
            'name': 'File Counts'
        },
        {
            'labels': ['SQL Dumps', 'CSV Files', 'Hash Files', 'Space Separated Files', 'Dox Files'],
            'values': file_sizes,
            'type': 'pie',
            'domain': {'x': [.52, 1]},
            'hole': .4,
            'hoverinfo': 'label+percent',
            'name': 'Disk Usage'
        }
    ],
    'layout': 
    {
        "title": "File Type and Size Distributions",
        "annotations": [
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "Count",
                "x": 0.20,
                "y": 0.5
            },
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "Disk",
                "x": 0.8,
                "y": 0.5
            }
        ]
    }
}

iplot(fig)

<h2>Entity Resolution</h2>
<p>Try to determine the number of effected persons exist across multiple database dumps, tables, etc.  To start, we're going to extract an xlsx file.</p>

In [ ]:
from openpyxl import load_workbook
wb = load_workbook(filename = base_dir+'runehq_db.xlsx')
sheet = wb.worksheets[0]
i = 0

rows = sc.parallelize([[cell.value for cell in row if cell.value is not None] for row in sheet.rows])

<h3>Read data from multiple sources</h3>

In [ ]:
def r2r(line, d_type, split):
    try:
        if d_type==0:
            tok = line.split(split)
            return Row(idd=tok[0], uname=tok[1], email=tok[2], hashed_pass=tok[3], ps_wd=tok[4])
        if d_type==1:
            tok = line.split(split)
            if tok[0]=='societa':
                return None
            return Row(
                societa=tok[0], 
                indirizzo=tok[1],
                citta=tok[2],
                cap=tok[3],
                provincia=tok[4],
                regione=tok[5],
                p_iva=tok[6],
                tel=tok[7].split()[1],
                fax=tok[8].split()[1],
                email=tok[9],
                nome=tok[10],
                cognome=tok[11]
            )
        if d_type==2:
            print line
            return Row(
                name=line[0],
                email=line[1],
                joined=line[2],
                ip_address=line[3],
                members_pass_hash=line[4],
                members_pass_salt=line[5]
            )
        if d_type==3:
            tok = line.split(split)
            return Row(idd=tok[0], email=tok[1], pswd=tok[2])
            
    except:
        return None


df0 = sc.textFile(base_dir+"ss_files/1.txt").map(lambda x: r2r(x, 0, " ")).filter(lambda x: x!=None).toDF()
df1 = sc.textFile(base_dir+"ss_files/4privatedbs.txt").map(lambda x: r2r(x, 0, " ")).filter(lambda x: x!=None).toDF()
df2 = sc.textFile(base_dir+"csv_files/Yahoo.txt").map(lambda x: r2r(x, 3, ":")).filter(lambda x: x!=None).toDF()
df3 = sc.textFile(base_dir+"csv_files/asus_customers_db.txt").map(lambda x: r2r(x, 1, "\t")).filter(lambda x: x!=None).toDF()
df4 = rows.map(lambda x: r2r(x, 2, "")).filter(lambda x: x!=None).toDF()

<h3>Get Individual DB Counts</h3>

In [ ]:
dsets = [df0, df1, df2, df3, df4]
for i in range(len(dsets)):
    print "Dataset {} has {} rows with {} unique email addrs.".format(i, dsets[i].count(), dsets[i].dropDuplicates(['email']).count())


In [ ]:
df0 = df0.dropDuplicates(['email'])
df1 = df1.dropDuplicates(['email'])
df2 = df2.dropDuplicates(['email'])
df3 = df3.dropDuplicates(['email'])
df4 = df4.dropDuplicates(['email'])

dsets = [df0, df1, df2, df3, df4]

<h3>Find collisions</h3>

In [ ]:
for i in range(len(dsets)):
    for ii in range(i+1,len(dsets)):
        joined = dsets[i].join(dsets[ii], dsets[i].email==dsets[ii].email, 'inner')
        print "Join of dataset {} and {} has {} email collisions".format(i, ii, joined.count())
        

<h3>Make Unique Datasets</h3>

In [ ]:
p_s = "/Users/jgartner/Desktop/xdata_hackathon/box/connect/"
df1.toPandas().to_csv(p_s+'d1.csv', sep='\t', encoding='utf-8')
df2.toPandas().to_csv(p_s+'d2.csv', sep='\t', encoding='utf-8')
df3.toPandas().to_csv(p_s+'d3.csv', sep='\t', encoding='utf-8')
df4.toPandas().to_csv(p_s+'d4.csv', sep='\t', encoding='utf-8')

In [ ]:
joined = dsets[2].join(dsets[4], dsets[2].email==dsets[4].email, 'inner').toPandas()
joined.head()

In [ ]:
df5 = spark.createDataFrame(base_dir+'CMU_extractionsDOX.csv')

In [ ]:
df5 = spark.createDataFrame(df)

In [ ]:
f0 = open(base_dir+"csv_files/asus_customers_db.txt", "r")
count = 0
for line in f0:
    words = line.split("\t")
    print len(words)
    if count == 0:
        print words
    count +=1
    if count == 10:
        break